In [16]:
# Install the Prediction Guard client
! pip install -U predictionguard

# Python Imports
import os
import json

import pandas as pd
from getpass import getpass

pg_access_token = getpass('Enter your Prediction Guard access token: ')
os.environ['PREDICTIONGUARD_TOKEN'] = pg_access_token
os.environ['PREDICTIONGUARD_URL'] = "https://intel.predictionguard.com"

import predictionguard as pg

# Transcripts
# Read the transcripts from the csv file named "cleaned_output_file"
transcripts = pd.read_csv('cleaned_output_file.csv')

# Convert the transcripts dataframe to a dictionary with id as the key and text as the value
transcripts = transcripts.set_index('id')['text'].to_dict()

# Test data
test = pd.read_csv('test.csv')

# Define a function to create an LLM prompt based on the question and the transcript
def create_prompt(question, transcript):
  # Use a template to create the prompt
  prompt = f"""### Instruction:
Answer the question based on the transcript.

### Question:
{question}

### Transcript:
{transcript}

### Response:
""".format(question=question, transcript=transcript)

  # Return the prompt
  return prompt

# Define a function to extract the answer from the LLM result
def extract_answer(result):
  # Use a simple heuristic to extract the answer from the result
  # Assume that the answer is the first line of the result
  # Check if the result dictionary has the "text" key and that it is not empty
  if "text" in result and result["text"] != "":
    answer = result["text"].split("\n")[0] # access the value of the "text" key and split it by the newline character
  else:
    # If the "text" key is not found or is empty, use a special token [UNK] to indicate that the answer is unknown or not applicable
    answer = "[UNK]"

  # Return the answer
  return answer

# Initialize an empty list to store the answers
answers = []

# Loop over all the rows in the test data
for _, row in test.iterrows():
  # Get the question and the transcript id from the row
  question = row['Question']
  transcript_id = row['Transcript']

  # Get the transcript text from the transcripts dictionary
  transcript = transcripts[transcript_id]

  # Create an LLM prompt based on the question and the transcript
  prompt = create_prompt(question, transcript)

  # Use the LLM model Llama-2-7B to generate a result based on the prompt
  result = pg.Completion.create(
      model="Falcon-40B-Instruct",
      prompt=prompt
  )

  # Extract the answer from the result
  answer = extract_answer(result)

  # Append the answer and the question id to the answers list
  answers.append([answer, row['Id']])

# Convert the answers list to a pandas dataframe
answers = pd.DataFrame(answers, columns=["Text", "Id"])

# Save the answers dataframe to a csv file
answers.to_csv("sample_submission_8nov.csv", index=False)


Enter your Prediction Guard access token: ········


ValueError: Could not make prediction. 